In [ ]:
import pandas as pd
data = pd.read_csv(r'C:\Users\misha\Desktop\total_dataset_to_ML.csv')
data

In [ ]:
#data = data[data['target'].isin([0, 1, 2, 4])]
data

In [ ]:
X = data.drop(['target'], axis=1)
X.head(3)

In [ ]:
y = data['target']
y.head(3)

In [ ]:
y.value_counts()

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25,random_state=42)

In [ ]:
y_train.value_counts()

In [ ]:
y_test.value_counts()

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score
from sklearn.metrics import recall_score
from datetime import datetime
from imblearn.over_sampling import SMOTE
from imblearn.pipeline import Pipeline

param_grid_svm = {
    'sampling__sampling_strategy': ['not majority'],
    'SVM__C': [0.1, 1],
    'SVM__gamma': [1, 0.1],
    'SVM__kernel': ['sigmoid', 'poly', 'rbf']
}

param_grid_knn = {
    'sampling__sampling_strategy': ['not majority'],
    'KNN__n_neighbors': [1,2,3,4,5,6,7,8,9,10,11,12]
}

param_grid_dt = {
    'sampling__sampling_strategy': ['not majority'],
    'criterion':  ['gini', entropy], 
    'DT__max_depth':  [8, 10, 12, 14],
    'DT__max_features': ['log2', 0.2, 0.3, 0.4],
    'splitter': ['best', 'random']
}

param_grid_rf = {
    'sampling__sampling_strategy': ['not majority'],
    'RF__n_estimators': [100, 300],
    'RF__max_features': ['sqrt', 0.2],
    'RF__max_depth': [8, 10, 12],
    'criterion': ['gini', 'entropy']
}

models = []
models.append(('SVM', SVC(), param_grid_svm))
models.append(('KNN', KNeighborsClassifier(), param_grid_knn))
models.append(('DT', DecisionTreeClassifier(), param_grid_dt))
models.append(('RF', RandomForestClassifier(), param_grid_rf))


for name, model, parameters in models:
    model = Pipeline([
        ('sampling', SMOTE()),
        (name, model),
    ])
    start_time = datetime.now()
    print(start_time)
    cv = StratifiedShuffleSplit(n_splits=5, test_size=0.25, random_state=42)
    model = GridSearchCV(estimator=model, param_grid=parameters, scoring='recall_macro', cv=cv, verbose=10, n_jobs=1)
    model.fit(X_train, y_train)
    print(name)
    print('best score: ', model.best_score_)
    print('best estimator: ', model.best_estimator_)
    pred = model.best_estimator_.predict(X_test)
    print(confusion_matrix(y_test, pred))
    print(classification_report(y_test, pred))
    print(name, datetime.now() - start_time)